In [4]:
from datasets import load_dataset

dataset = load_dataset("openbookqa", "additional", split=["train", "test"])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [5]:
dataset[0][0]

{'id': '7-980',
 'question_stem': 'The sun is responsible for',
 'choices': {'text': ['puppies learning new tricks',
   'children growing up and getting old',
   'flowers wilting in a vase',
   'plants sprouting, blooming and wilting'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'D',
 'fact1': 'the sun is the source of energy for physical cycles on Earth',
 'humanScore': 1.0,
 'clarity': 2.0,
 'turkIdAnonymized': 'b356d338b7'}

In [6]:
def get_prompt(dataset, index, perm=[0, 1, 2, 3], has_choice=False):
    json_line = dataset[index]
    question = json_line["question_stem"]
    choices = json_line["choices"]
    choice_texts = choices["text"]
    choice_texts = [choice_texts[perm[i]] for i in range(len(choice_texts))]
    candidates = " ".join(
        [
            f"({label if has_choice else ' '}) {text}"
            for text, label in zip(choice_texts, choices["label"])
        ]
    ).replace("\n", " ")
    answer_key = json_line["answerKey"]
    answer_key_idx = ord(answer_key[0]) - ord("A")
    answer_text = choices["text"][answer_key_idx]
    prompt = f"Context: {json_line['fact1']}. Question: {question} \\n {candidates}"
    return prompt, answer_text

In [7]:
from tqdm import tqdm
import pickle
DATASET_NAME='obqa_fact'
for index, dataname in enumerate(['train','test']):
    container=[]
    for ques_index, ques in enumerate(tqdm(dataset[index])):
        container.append(
                get_prompt(dataset[index], ques_index, has_choice=False)
            )
    pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 500/500 [00:00<00:00, 3631.69it/s]


In [52]:
import json
import pickle
obj = pickle.load(open("test_with_abcd.pkl", "rb"))

with open("test_with_abcd.txt", "w") as f:
    json.dump(obj, f, indent=2)


In [3]:
# def get_prompt(index, perm):
#     json_line = dataset["test"][index]
#     question = json_line["question_stem"]
#     choices = json_line["choices"]
#     choice_texts = choices["text"]
#     choice_texts = [choice_texts[perm[i]] for i in range(len(choice_texts))]

#     def change(text, leng=20, is_question=False):
#         pad_x = [0] * leng
#         encoded = tokenizer.encode(text)[:-1]
#         if len(encoded) > leng:
#             print("too long ", len(encoded), leng, is_question)
#         pad_x[: len(encoded)] = encoded
#         return tokenizer.decode(pad_x)

#     candidates = " ".join(
#         [
#             change(f"( ) {text}", ANSWER_PAD)
#             for text, label in zip(choice_texts, choices["label"])
#         ]
#     ).replace("\n", " ")
#     # print(json_line)
#     answer_key = json_line["answerKey"]
#     answer_key_idx = ord(answer_key[0]) - ord("A")
#     # answer_text = choice_texts[answer_key_idx]
#     # id = "OBQA_" + json_line['id']
#     question_pad = f"{question} \\n "
#     prompt = f"{change(question_pad,QUESTION_PAD, is_question=True)}{candidates}"
#     return prompt, choice_texts